# Задание 1
Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Можно считать в google sheets, екселе, питоне или на листочке. Сделайте скрин получившейся таблицы и загрузите картинку на гитхаб, вставьте ссылку в поле ниже.

https://github.com/deniskapel/Compling_course/blob/master/Tfidf_table.PNG

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [2]:
pd.set_option('display.max_colwidth', 500)

In [3]:
class DatasetVectorizer():
    
    """ a class to vectorize a pandas dataset using one of sklearn vectorizers """
    def __init__(self, vec):
        self.vectorizer = vec
        self.data = pd.DataFrame()
        self.train = pd.DataFrame()
        self.test = pd.DataFrame()
        self.X = 0
        self.X_test = 0
        self.y = 0
        self.y_test = 0
        
    def load_data(self, path):
        """ opens a .csv file using pandas dataframe """
        self.data = pd.read_csv(path)
        
    def train_test_split(self):
        """ splits data using sklearn train_test_split() function """
        self.train, self.test = train_test_split(self.data, test_size=0.1, shuffle=True)
        self.train.reset_index(inplace=True)
        self.test.reset_index(inplace=True)
        
    def vectorizeXy(self, feature_col, value_col):
        """ transforms a dataset into X, X_test, Y and Y_test """
        self.X = self.vectorizer.fit_transform(self.train[feature_col])
        self.X_test = self.vectorizer.transform(self.test[feature_col])
        self.y = self.train[value_col].values
        self.y_test = self.test[value_col].values

In [4]:
dv = DatasetVectorizer(TfidfVectorizer(min_df=3, max_df=0.5))
dv.load_data('data/labeled.csv')
dv.data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n",1.0


# Задание 2. 
Векторизовать можно любым способом, но постарайтесь, чтобы 1 и 0 не получались в близостях.
Результат запишите в тетрадке.


### а) Посчитайте близость между 3 и 12666 текстами в датасете (labeled.csv из семинара) 

In [5]:
# vectorize whole dataset
X = dv.vectorizer.fit_transform(dv.data.comment)
dv.data.loc[[3, 12666]]

,comment,toxic
3,"Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n",1.0
12666,"Это не параноик, это дебил\n",1.0


In [6]:
print(1 - cosine_distances(X[3], X[12666]))
print(cosine_similarity(X[3], X[12666]))

[[0.5290017]]
[[0.5290017]]


### б) найдите 3 самых близких текста к тексту номер 43; выведите сами тексты и значения близостей, а не только индексы этих текстов. 

In [7]:
closest = cosine_distances(X[43], X)
print(closest.argsort()[0,:4])

[   43  1986 14325  5564]


In [8]:
# how close
print(1-closest[0,[43, 1986, 14325,  5564]])

[1.         0.20875876 0.20196174 0.1772219 ]


In [9]:
dv.data.loc[[43, 1986, 14325,  5564]]

,comment,toxic
43,"Люди зажрались и охуели если по мнению этих игроков андромеда лучше Антема. Хотя че там с багаутом76 сравнивают вон... Вот оно че оказывается, игроки просто охуели, ну ок. Вот долбанные пидерасы, не хотят покупать такую отличную игру, а еще смеют ругать такой божественный геймплей. Баги, хуевый геймплей поправят, а контент, ну его запилят, через год другой, причем бесплатно! Ага. Стоит лишь потерпеть!\n",1.0
1986,НУ И КАКАЯ МРАЗЬ КИДАЕТ ССЫЛКИ? ОХУЕЛИ ТАМ В КРАЙ УЖЕ?\n,1.0
14325,"Ну оль, ну вот че тебе надо? Иди в по сри, а этот достопочтенный тред оставь в покое\n",1.0
5564,"МАААМ, НУ ЧЕ ОНИ ЗА МНОЙ ХОДЯТ?\n",1.0


# Задание 3

Обучите 2 любых разных классификатора из семинара.

Предскажите токсичность для текстов из 2ch_corpus.txt (второй семинар) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные? *Можете предсказывать не все данные, но не менее 2500.

In [10]:
dv = DatasetVectorizer(TfidfVectorizer(min_df=5, max_df=0.4))
dv.load_data('data/labeled.csv')
dv.train_test_split()
dv.vectorizeXy('comment', 'toxic')

In [11]:
dvach = open('data/2ch_corpus.txt', encoding="utf-8").readlines()[0:2500]
dvach_vec = dv.vectorizer.transform(dvach)
dvach_vec.shape

(2500, 7604)

### LogisticRegression

In [12]:
lr = LogisticRegression(C=1, class_weight='balanced')
lr.fit(dv.X, dv.y) # create a model
preds = lr.predict(dv.X_test)
print(classification_report(dv.y_test, preds))

              precision    recall  f1-score   support

         0.0       0.90      0.87      0.88       938
         1.0       0.77      0.81      0.79       504

    accuracy                           0.85      1442
   macro avg       0.83      0.84      0.84      1442
weighted avg       0.85      0.85      0.85      1442



In [15]:
probas = lr.predict_proba(dv.X_test)
probas

array([[0.844838  , 0.155162  ],
       [0.84355221, 0.15644779],
       [0.4727683 , 0.5272317 ],
       ...,
       [0.07061404, 0.92938596],
       [0.66437344, 0.33562656],
       [0.83338751, 0.16661249]])

In [14]:
# check if probability of class 2 is higher than a given threshold
preds = (probas[:,1]>0.60).astype(int) 
print(classification_report(dv.y_test, preds))

              precision    recall  f1-score   support

         0.0       0.84      0.95      0.89       938
         1.0       0.87      0.66      0.75       504

    accuracy                           0.85      1442
   macro avg       0.85      0.80      0.82      1442
weighted avg       0.85      0.85      0.84      1442



In [16]:
dvach_preds_lr = lr.predict(dvach_vec)
dvach_probas_lr = lr.predict_proba(dvach_vec)
print(dvach_preds_lr.shape, dvach_probas_lr.shape)

(2500,) (2500, 2)


In [17]:
dvach_lr_labeled = pd.DataFrame({"comment": dvach, 'toxic': dvach_preds_lr, 'prob_tox' : dvach_probas_lr[:, 1]})

### DecisionTreeClassifier

In [18]:
dt = DecisionTreeClassifier(max_depth=100, class_weight='balanced')
dt.fit(dv.X, dv.y)
preds = dt.predict(dv.X_test)

print(classification_report(dv.y_test, preds))

              precision    recall  f1-score   support

         0.0       0.83      0.70      0.76       938
         1.0       0.57      0.74      0.64       504

    accuracy                           0.71      1442
   macro avg       0.70      0.72      0.70      1442
weighted avg       0.74      0.71      0.72      1442



In [21]:
probas = dt.predict_proba(dv.X_test)
probas

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [20]:
# check if probability of class 2 is higher than a given threshold
preds = (probas[:,1]>0.5630).astype(int) 
print(classification_report(dv.y_test, preds))

              precision    recall  f1-score   support

         0.0       0.83      0.70      0.76       938
         1.0       0.57      0.74      0.64       504

    accuracy                           0.71      1442
   macro avg       0.70      0.72      0.70      1442
weighted avg       0.74      0.71      0.72      1442



Предскажите токсичность для текстов из 2ch_corpus.txt (второй семинар) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?
*Можете предсказывать не все данные, но не менее 2500.

In [22]:
dvach_preds_dt = dt.predict(dvach_vec)
dvach_probas_dt = dt.predict_proba(dvach_vec)
print(dvach_preds_dt.shape, dvach_probas_dt.shape)

(2500,) (2500, 2)


In [23]:
dvach_dt_labeled = pd.DataFrame({"comment": dvach, 'toxic': dvach_preds_dt, 'prob_tox' : dvach_probas_dt[:, 1]})

Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

In [24]:
dvach_lr_labeled.sort_values('prob_tox',ascending=False).head(10)

,comment,toxic,prob_tox
87,"Ты превращаешься в Суисейсеки, десу.\n",1.0,0.999150
271,>ElixirНу ты и говноед.\n,1.0,0.999150
2463,Крысу ты разделовал?\n,1.0,0.999150
118,оуоуоуо уча уча уча аиавивива ну ты ну ты ну ты ВИЧУУУУУУУУУУУУУУУУУХА\n,1.0,0.996437
938,Ебать ты юморист донный.\n,1.0,0.995942
1775,"Кто подданный? Ты подданый, ёпт. Здравствуй.\n",1.0,0.993945
2036,Ты зацени какой подклад на пиджаке\n,1.0,0.993906
1489,И нахуй ты это ему написал?\n,1.0,0.992868
1760,Просто ты скучный и предсказуемый.\n,1.0,0.992781
898,Ты просто феерический счетовод.\n,1.0,0.992781


In [25]:
dvach_dt_labeled.sort_values('prob_tox',ascending=False).head(10)

,comment,toxic,prob_tox
1250,"Я понимаю, что тебя пидорасит, это нормально. У самого пару раз бывало. Сиди дома, попробуй уснуть. Тебе врачи то же самое скажут.\n",1.0,1.0
1397,Сколько по времени? Нервная ткань же не восстанавливается?\n,1.0,1.0
2246,"Сказал двачер, который сдохнет в 30 лет в нищете от инфаркта\n",1.0,1.0
678,">сцука >сц> убейся апстенку кококо Шло бы ты в свои контакты, хуйло из 2005\n",1.0,1.0
680,"ты дибил 76 мест в обеих кружках на 56 человек, только 56 могут ходить в один кружок остальные 20 в 2 кружка одновременно, если нет места лишнего ни для кого потому что по условию сцука 28 и 48 \n",1.0,1.0
1957,"Будь я крутым парнем, то тоже бы Дорна слушал.\n",1.0,1.0
682,"Учеников всего 56. 28 ходят на шахматы, значит 56-28=28 - точно не ходят на шахматы. 48 ходят на танцы, значит 56 - 48 = 8 - точно не ходят на танцы. 28+8=36 - кол-во, которое точно посещает один из двух кружков, но не два одновременно. 56-36 = 20 - кол-во учеников, которые ходят на два кружка одновременно.\n",1.0,1.0
683,"Иди на хуй дурбецело. В задаче нет этих условий, а значит ходят все.\n",1.0,1.0
1403,"Скорость выведения токсинов и их захват. Вообще по-разному, на вкус и цвет. Мне привычнее глотать таблетки, чем гель в воде.\n",1.0,1.0
1401,"Живот болит+температура+лейкоцитоз=эмоциональный стресс, ну ты и еблааан (хирург кун)\n",1.0,1.0


Пересечений в топ-10 не надблюдается, предсказание чуть лучше работает на модели линейной регрессии.